# WAVE tutorial: a smart home

WAVE is platform for delegable authorization. We are going to test it out by looking at some use cases in a smart home context. You can also use WAVE for securing other things, like APIs, but this is a fun tutorial to cover the basics.

## Getting started

The lines below will "boot" your house. There is a home server that acts as the root of authority for your house and all its devices. Give your house a unique nickname, as later you will be accessing data from other people's houses. 

In [ ]:
from tut import *
my_unique_nickname="john-smith2"
# wave is a connection to the WAVE daemon, that you will use for authorization actions
# homeserver is the control interface to your home server, allowing you to request permissions etc.
# mqtt is a connection to the rise camp MQTT server that routes all commands and sensor information 
wave, homeserver, mqtt = Initialize(my_unique_nickname)

## Creating your entity

An *entity* represents a person or device. Your home server has an entity which is the root of permissions in the house, but every occupant of the house also needs their own entity so they can be granted permissions to interact with the house. Go ahead and create your entity by running the command below. 

Note that usually the command would be `entity = wave.CreateEntity(wv.CreateEntityParams())` but to make sure you don't lose your entity if you restart the notebook, we have made a wrapper that saves it to disk

In [ ]:
entity, _ = createOrLoadEntity(wave, "myEntity")

# a perspective is the form of an entity you use in calls to WAVE
perspective = wv.Perspective(entitySecret=wv.EntitySecret(DER=entity.SecretDER))

## Proofs of permission

Let's try control the house with this brand new entity. To do so, we need to form a *proof of permissions*:


In [ ]:
lightproof = wave.BuildRTreeProof(wv.BuildRTreeProofParams(
    # our secret entity perspective
    perspective=perspective,
    # the domain of authority we are accessing
    namespace=homeserver.namespace(),
    # what we want to prove
    statements=[
        wv.RTreePolicyStatement(
            permissionSet=smarthome_pset,
            permissions=["write"],
            resource="smarthome/light/control",
        )
    ]
))

# lets check if that worked:
if lightproof.error.code != 0:
    print ("Proof building failed:", lightproof.error.message)
else:
    print ("Proof building succeeeded")

If this is the first time you've run the notebook, the above will fail. This is because this newly created entity has not been granted any permissions! Why should it be able to control the house? In real life, you would now communicate with a person/service that has the permissions you want and give them your entity hash. With this, they can grant you permissions. We will emulate that with a simple function call:

In [ ]:
homeserver.grant_permissions_to(entity.hash)

That grants you permissions to everything within the house, later we will do a permission grant from scratch, so you can see how it is done. For now, lets try building a proof again:

In [ ]:
lightproof = wave.BuildRTreeProof(wv.BuildRTreeProofParams(
    # our secret entity perspective
    perspective=perspective,
    # the domain of authority we are accessing
    namespace=homeserver.namespace(),
    # Tell WAVE to look for new permissions
    resyncFirst=True,
    # what we want to prove
    statements=[
        wv.RTreePolicyStatement(
            permissionSet=smarthome_pset,
            permissions=["write"],
            resource="smarthome/light/control",
        )
    ]
))

if lightproof.error.code != 0:
    print ("Proof building failed:", lightproof.error.message)
else:
    print ("Proof building succeeeded")

This should succeed. We can now attach this proof to a command we send to the light, and it will obey the command because it can verify we are authorized. For this demo, the smart home devices rendered at the top of the notebook are listening for commands on an MQTT topic unique to your nickname.

In [ ]:
mqtt.publish(my_unique_nickname+"/smarthome/light/control",
#                      you can also use {"state":"off"}
                  composeMessage(lightproof, {"state":"on"}))

If you scroll to the top of the notebook you should see that the light has turned on. To make it easier to see, you can add a copy of your smart home devices here:

In [ ]:
homeserver.render()

In addition to controlling lights, you can also form a proof for writing to the message box:

In [ ]:
msgproof = wave.BuildRTreeProof(wv.BuildRTreeProofParams(
    perspective=perspective,
    namespace=homeserver.namespace(),
    resyncFirst=True,
    statements=[
        wv.RTreePolicyStatement(
            permissionSet=smarthome_pset,
            permissions=["write"],
            resource="smarthome/notify",
        )
    ]
))

if msgproof.error.code != 0:
    print ("Proof building failed:", msgproof.error.message)
else:
    print ("Proof building succeeeded")


You send messages to it the same way you do for the light:

In [ ]:
mqtt.publish(my_unique_nickname+"/smarthome/notify",
                  composeMessage(msgproof, "hello world"))

Now that you have sent some authorization proofs attached to messages, let's walk through how you would decrypt and validate a message that you receive. Let's subscribe to the thermostat report topic and make the light mirror the occupancy report from the thermostat:

In [ ]:
def thermostat_cb(msg):
    # first decrypt the message
    resp = wave.DecryptMessage(wv.DecryptMessageParams(
        perspective= perspective,
        ciphertext= msg.payload,
        resyncFirst= True))
    if resp.error.code != 0:
        print ("dropping thermostat message:", resp.error.message)
        return
    # then break it up into proof + body
    proof, body = decomposeMessage(resp.content)
    
    # now validate the proof
    resp = wave.VerifyProof(wv.VerifyProofParams(
        proofDER=proof,
        requiredRTreePolicy=wv.RTreePolicy(
            namespace=homeserver.namespace(),
            statements=[wv.RTreePolicyStatement(
                permissionSet=smarthome_pset,
                permissions=["write"],
                resource="smarthome/thermostat/report",
            )]
        )
    ))
    if resp.error.code != 0:
        print ("dropping message: ", resp.error.message)
        return
        
    # the proof is valid!
    house_occupied = body["occupied"]
    # fill in some code here to turn the light on if the house is occupied, and off otherwise
    
    
mqtt.subscribe(my_unique_nickname+"/smarthome/thermostat/report", thermostat_cb)

You have now created a controller that consumes secure feeds and produces a control feed!

To disable the controller above, put a `return` at the top of the callback function and re-run the cell.

## Interacting across namespaces

You are a good neighbor and you like to help out. Your neighbor is away on vacation, so you want to tie their house lights to yours so that their house seems occupied.

First off, you need to find a partner (like the person next to you!) and you need to communicate your entity hash to them. The easiest way is to use base64 encode the hash and use mqtt:

In [ ]:
import base64
# some utility functions
def hashToBase64(hash):
    return str(base64.b64encode(hash), "utf8")
def hashFromBase64(b64):
    return base64.b64decode(b64)

# print out messages from people looking for partners
def looking_for_partner(msg):
    # you can comment this out and re-run the cell to turn off these messages if they are annoying
    print ("looking for partner > "+str(msg.payload,"utf8"))
    pass
mqtt.subscribe("looking-for-partner", looking_for_partner)
mqtt.publish("looking-for-partner", 
             "I am "+my_unique_nickname+
             " my entity is "+hashToBase64(entity.hash)+ 
             " and my home namespace is "+hashToBase64(homeserver.namespace()))

Get your neighbor to run the above as well, and copy their hash and nickname



In [ ]:
partner_hash = hashFromBase64( paste here )
partner_home_namespace = hashFromBase64 ( paste here)
partner_nickname = "paste here"

Decide between you and your partner who is going on vacation (their lights will be controlled) and who is doing the controlling.

## Having your lights controlled:

You need to grant the controlling entity (stored in `partner_hash`) permission to control your lights:

In [ ]:

wave.CreateAttestation(wv.CreateAttestationParams(
    perspective=perspective,
    subjectHash=partner_hash,
    publish=True,
    policy=wv.Policy(rTreePolicy=wv.RTreePolicy(
        namespace=homeserver.namespace(),
        indirections=5,
        statements=[
            wv.RTreePolicyStatement(
                permissionSet=smarthome_pset,
                permissions=["write"],
                resource="smarthome/light/control",
            )]
    ))))


Note that what you have just done is *delegation*: you received some permissions from the home server entity and you are passing on a subset of those permissions on to your partner.

## Controlling your partner's lights:

You need to build a proof that you can control your partner's light:

In [ ]:
partnerlightproof = wave.BuildRTreeProof(wv.BuildRTreeProofParams(
    perspective=perspective,
    # the namespace will be our partner's home, not ours
    namespace=partner_home_namespace,
    # Tell WAVE to look for new permissions
    resyncFirst=True,
    # what we want to prove
    statements=[
        wv.RTreePolicyStatement(
            permissionSet=smarthome_pset,
            permissions=["write"],
            resource="smarthome/light/control",
        )
    ]
))

if partnerlightproof.error.code != 0:
    print ("Proof building failed:", partnerlightproof.error.message)
else:
    print ("Proof building succeeeded")

Now you can use this proof to publish a message:


In [ ]:
mqtt.publish(partner_nickname+"/smarthome/light/control", composeMessage(partnerlightproof, {"state":"on"}))

By referring to the thermostat controller above, create a new controller that reads your own light state  `nickname/smarthome/light/report` and controls your partner's light `partnernickname/smarthome/light/control`.

In [ ]:
# create callback function
#    decrypt our light state
#    verify proof
#    publish control message using partnerlightproof

# subscribe to our light state

## Bonus section: decrypting your partner's streams

Your vacation-happy neighbor is having a problem with package theft. He has asked you to create a controller that tells you when he gets a package and he is not home. Everyone in the neighborhood has a smart home with a motion sensor by the front door. They also have thermostats that have occupancy sensors. If there is motion in front of the neighbor's house, and nobody is home, then maybe you should go check for a package.

Discuss with your partner who is going on vacation, and who is the kind neighbor picking up packages:

## Going on vacation

You will need to grant your partner the ability to decrypt your thermostat data (which tells them if the house is occupied by the house sitter or not) and the ability to decrypt the motion sensor data (which tells them a package was delivered.

In [ ]:

# grant your partner the ability to decrypt your thermostat data
wave.CreateAttestation(wv.CreateAttestationParams(
    perspective=perspective,
    subjectHash=partner_hash,
    publish=True,
    policy=wv.Policy(rTreePolicy=wv.RTreePolicy(
        namespace=self.ent.hash,
        indirections=5,
        statements=[
            wv.RTreePolicyStatement(
                permissionSet=wv.WaveBuiltinPSET,
                permissions=[wv.WaveBuiltinE2EE],
                resource="smarthome/thermostat/report",
            )]
    ))))

# and grant them the ability to decrypt your motion sensor
wave.CreateAttestation(wv.CreateAttestationParams(
    perspective=perspective,
    subjectHash=partner_hash,
    publish=True,
    policy=wv.Policy(rTreePolicy=wv.RTreePolicy(
        namespace=self.ent.hash,
        indirections=5,
        statements=[
            wv.RTreePolicyStatement(
                permissionSet=wv.WaveBuiltinPSET,
                permissions=[wv.WaveBuiltinE2EE],
                resource="smarthome/motion/report",
            )]
    ))))


## The kind neighbor

To check you have permissions, lets try building a proof that you can decrypt the motion sensor data:


In [ ]:
decryptmotionproof = wave.BuildRTreeProof(wv.BuildRTreeProofParams(
    perspective=perspective,
    # the namespace will be our partner's home, not ours
    namespace=partner_home_namespace,
    # Tell WAVE to look for new permissions
    resyncFirst=True,
    # what we want to prove
    statements=[
        wv.RTreePolicyStatement(
            permissionSet=wv.WaveBuiltinPSET,
            permissions=[wv.WaveBuiltinE2EE],
            resource="smarthome/motion/report",
        )
    ]
))

if decryptmotionproof.error.code != 0:
    print ("Proof building failed:", decryptmotionproof.error.message)
else:
    print ("Proof building succeeeded")

Now, by consulting the subscription callbacks you have written in the past, write a controller that listens to and decrypts your neighbors motion and occupancy data, and publishes a notification message to your own homeserver if you need to check for a package.